# 06、自定义数据集
光用API加载数据我觉得不太行，只能自己自娱自乐跑跑例子，前几节稍微提到过数据集的定义，这一节对如何定义数据集进行详细的分析和讲解。


Pytorch有很高的灵活性，你完全可以自己从零开始，使用numpy构建数据集，比如我进行一个for循环读取数据然后使用模型进行计算，或者自己定义一个类啥的都是很简单的，但是为了方便我觉得还是遵循pytorch的建议来继承某些类来进行数据集生成。

In [63]:
#  首先我可以使用numpy新建一些数据，直接使用网络进行训练，为了简单我使用LeNet
import torch
import torch.nn as nn
# 假装图片
images = []
labels = []
for i in range(10):
    images.append(torch.rand(10, 1, 28, 28))
    labels.append(torch.randint(0, 2, [10]))
print(labels)

[tensor([0, 1, 0, 1, 0, 1, 1, 1, 1, 1]), tensor([1, 0, 0, 0, 1, 0, 1, 1, 1, 1]), tensor([0, 0, 0, 0, 1, 0, 0, 1, 0, 1]), tensor([1, 1, 1, 0, 0, 1, 0, 1, 0, 0]), tensor([0, 0, 0, 1, 1, 1, 0, 1, 0, 0]), tensor([0, 1, 1, 1, 0, 1, 1, 1, 0, 1]), tensor([0, 1, 1, 1, 1, 1, 0, 1, 1, 1]), tensor([1, 1, 0, 0, 0, 0, 1, 0, 1, 1]), tensor([1, 0, 1, 1, 1, 0, 0, 1, 1, 1]), tensor([1, 0, 1, 1, 1, 1, 1, 0, 0, 0])]


In [18]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(     #input_size=(1*28*28)
            nn.Conv2d(1, 6, 5, 1, 2), #padding=2保证输入输出尺寸相同
            nn.ReLU(),      #input_size=(6*28*28)
            nn.MaxPool2d(kernel_size=2, stride=2),#output_size=(6*14*14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),      #input_size=(16*10*10)
            nn.MaxPool2d(2, 2)  #output_size=(16*5*5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)

    # 定义前向传播过程，输入为x
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # nn.Linear()的输入输出都是维度为一的值，所以要把多维度的tensor展平成一维
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [20]:
model =  LeNet()
certion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
model

LeNet(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): ReLU()
  )
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [60]:
for i,l in zip(images, labels):
    optimizer.zero_grad()
    out = model(i)
    loss = certion(out, l)
    loss.backward()
    optimizer.step()
    print(f'loss:{loss.item()}, avg_loss{loss.item() * i.size(0)}')
#     _, pre = out.max(1)
#     acc = (pre == l).sum().item()
#     print(acc)
#     acc_ = acc / i.size(0)
#     print(acc_)

loss:0.6814153790473938, avg_loss3.407076895236969
loss:0.8256223797798157, avg_loss4.128111898899078
loss:0.7207103967666626, avg_loss3.603551983833313
loss:0.6993415355682373, avg_loss3.4967076778411865
loss:0.6882819533348083, avg_loss3.4414097666740417
loss:0.8258719444274902, avg_loss4.129359722137451
loss:0.5788917541503906, avg_loss2.894458770751953
loss:0.8512927889823914, avg_loss4.256463944911957
loss:0.7554869651794434, avg_loss3.777434825897217
loss:0.7351841926574707, avg_loss3.6759209632873535


这里看到你可以使用torch、numpy新建一些数据或者处理一些数据来进行训练还是很简单的，但是呢随着数据量的增长这样写入内存的方式好像有点不太行所以你需要一个迭代器类来处理，假如你的python比较厉害可以写一个可迭代的类来处理数据，比如像下面一样

In [77]:
class Mydata:
    def __init__(self, data):
        self.data = data
        # 可以在这里进行简单处理
    def __getitem__(self, index):
        return self.data[index]
for i in Mydata(images[0]):
    print(i.shape)
    # 然后就可以在这进行训练啥的，按需生成绝不浪费
    break

torch.Size([1, 28, 28])


然后我们又意识到，一个一个数据训练好像效果不太好，而且浪费了计算机CPU，GPU的算力，能不能一次处理很多数据比如一次计算100张图片，显然是可以你可以自己写点代码实现这个功能，为了方便我们直接使用pytorch自带的数据生成器，来对数据进行批次获取。

- trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,shuffle=True)

使用torch.utils.data.DataLoader就可以对数据进行获取批次的操作其中第一个参数是传入数据生成器，是一个可迭代对象，然后第二个就是批次大小，shuffle是是否打乱数据。

In [79]:
class Mydata:
    def __init__(self, data):
        self.data = data
        # 可以在这里进行简单处理
    def __getitem__(self, index):
        return self.data[index]
    def __len__(self):
        return len(self.data)

# 我们的images有10个数据我们试试 用API一次产生两个图片的批次
data_ = Mydata(images[0])
loader = torch.utils.data.DataLoader(data_, batch_size=2,shuffle=True)
for i in loader:
    print(i.shape)
    break
# 看到输出就是2,1,28,28了，挺好，按需生成数据

torch.Size([2, 1, 28, 28])


我们又看到上一节数据生成的时候的数据为什么可以transform，对数据进行处理，这里我们介绍最后一个方法，新建类继承torch.utils.data.Dataset类然后重写len和getitem方法即可，len是统计数据有多长的，改进一下我们数据类。

In [90]:
from torch.utils.data import Dataset
import numpy as np
from torchvision.transforms import transforms
# 为了验证transform的作用，我就用numpy产生数据。
data = np.random.rand(10, 1, 28, 28)

In [92]:
class Mydata(Dataset):
    def __init__(self, data, transform=None):
        super(Mydata).__init__()
        self.data = data
        self.transform = transform
        # 可以在这里进行简单处理
    def __getitem__(self, index):
        # 进行一个判断，看看是不是需要做transform
        sample = self.data[index]
        if self.transform:
            sample = self.transform(self.data[index])
        return sample
    def __len__(self):
        return len(self.data)

transform = transforms.Compose([
    transforms.ToTensor(),

])
# 我们的images有10个数据我们试试 用API一次产生两个图片的批次
data_ = Mydata(data, transform=transform)
loader = torch.utils.data.DataLoader(data_, batch_size=2,shuffle=True)
for i in loader:
    print(type(i))
    break


<class 'torch.Tensor'>


我们生成的是numpy数据 但是我们调用 了transform方法将其转换为tensor格式，可以验证transform可以正常工作，至此我们自己创建数据集的基本用法结束了，遇到更复杂的问题根据基本用法见招拆招即可。